In [1]:
import pandas as pd
df_train=pd.read_csv("train.csv")

In [2]:
# Separate features and target
X_train = df_train.drop(columns=["label"])
y_train = df_train["label"]

In [3]:
pip install bayesian-optimization


Note: you may need to restart the kernel to use updated packages.


In [4]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [5]:
def objective_bo(max_depth, min_samples_split, min_samples_leaf, criterion):
    criterion = 'gini' if criterion < 0.5 else 'entropy'
    clf = DecisionTreeClassifier(
        max_depth=int(max_depth),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        criterion=criterion,
        random_state=42
    )
    score = cross_val_score(
        clf, X_train, y_train,
        scoring='accuracy',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    ).mean()
    return score

pbounds = {
    'max_depth': (5, 60),
    'min_samples_split': (2, 10),
     'min_samples_leaf': (1, 5),
    'criterion': (0, 1)
}


In [6]:
optimizer = BayesianOptimization(f=objective_bo, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=30, n_iter=80)

print("Best parameters:", optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | criterion |
-------------------------------------------------------------------------
| 1         | 0.9348418 | 25.599706 | 9.6057144 | 3.9279757 | 0.5986584 |
| 2         | 0.8828101 | 13.581025 | 3.2479561 | 1.2323344 | 0.8661761 |
| 3         | 0.9389635 | 38.061325 | 7.6645806 | 1.0823379 | 0.9699098 |
| 4         | 0.9390608 | 50.784345 | 3.6987128 | 1.7272998 | 0.1834045 |
| 5         | 0.9236169 | 21.733323 | 6.1980514 | 2.7277800 | 0.2912291 |
| 6         | 0.9332563 | 38.651909 | 3.1159508 | 2.1685785 | 0.3663618 |
| 7         | 0.9385331 | 30.083849 | 8.2814076 | 1.7986951 | 0.5142344 |
| 8         | 0.9332611 | 37.582801 | 2.3716033 | 3.4301794 | 0.1705241 |
| 9         | 0.7941492 | 8.5778376 | 9.5910842 | 4.8625281 | 0.8083973 |
| 10        | 0.9228995 | 21.753757 | 2.7813769 | 3.7369321 | 0.4401524 |
| 11        | 0.8570021 | 11.712102 | 5.9614152 | 1.1375540 | 0.9093204 |
| 12        | 0.9272183 | 19.232898 | 

In [7]:
best_dt_BOGP=optimizer.max

In [8]:
import joblib

In [9]:
best_dt_BOGP=best_dt_BOGP['params']

In [10]:
best_dt_BOGP['criterion']='entropy'

In [11]:
# Convert numeric parameters to int
best_dt_BOGP = {k: int(v) if isinstance(v, (np.integer, np.floating)) else v for k, v in best_dt_BOGP.items()}

In [12]:
joblib.dump(best_dt_BOGP, "best_dt_BOGP.pkl")

['best_dt_BOGP.pkl']